In [21]:
# 0. API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API 키 정보 로드
load_dotenv(dotenv_path='C:/Users/user/Desktop/RAG-Implementation/CSV RAG implementation/.env')
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [22]:
'''1. INDEXING'''
# 1. 문서 로드 (CSV 파일)
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV loader 생성
loader = CSVLoader(
    file_path='C:/Users/user/Downloads/한국동서발전(주)_대기권 밖 일사량_20240125.csv',
    encoding='cp949'
)

# 데이터 로드
docs = loader.load()

# 데이터 양 확인
print(len(docs))

# 불러온 첫번째 row 데이터 확인
print(docs[0].page_content)

78840
일자: 01-01 00:00
지역: 광주
위도: 35.1729
경도: 126.8916
대기권 밖 일사량: 0


In [23]:
'''1 INDEXING'''
# 2. 문서 SPLIT 
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0 # (row 간 overlap은 불필요)
)

splits = text_splitter.split_documents(docs)
len(splits)

78840

In [24]:
# gpu 사용가능여부 확인
# cpu 사용시 large csv 파일을 읽을때 오래 걸림

'''
False일시:
1. 기존 버전 삭제:
pip uninstall torch torchvision torchaudio

2. CUDA 지원 버전 설치 (12.1 버전 기준):
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
'''

import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

2.12.0.dev20260218+cu128
True
NVIDIA GeForce RTX 5060 Ti
12.8


ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434

'''
오류 원인 요약
PyTorch 2.6 버전부터 torch.load() 기본 옵션이 **weights_only=True**로 바뀌었습니다.
기존 .pt 파일은 모델 가중치(state_dict)와 함께 args 객체 (argparse.Namespace) 도 저장되어 있습니다.
그런데 Namespace는 pickle로 저장된 Python 객체인데,
weights_only=True일 때는 이런 "비순수" 객체들을 로드할 수 없습니다.
해결 방법은 간단합니다. 아래와 같이 torch.load를 했다면

checkpoint = torch.load(model_path)
 

weights_only 버전에 False를 입력하여 예전처럼 모든 데이터를 읽게 합니다.

checkpoint = torch.load(model_path, weights_only=False)
 

그럼 오류가 해결됩니다.
'''

이런 메세지가 뜬다면 이전 gpu 사용가능여부 확인 코드에서 버전 체크:

2.6 이하라면: ex. 2.5.1+cu121
PyTorch 최신 버전 2.6.0 이상으로 업데이트 필요함

현재 torch uninstall
pip uninstall torch torchvision torchaudio -y


새로운 torch install 자신의 컴퓨터에 맞는 GPU로 설정해야함 ^ 윗코드 참조
pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

버전이 2.12.0.... 인데 왜 돌아가는가?
2.12.0.dev20260218+cu128
 │      └─ 2026-02-18 빌드된 개발버전
 └─ 안정화 버전 기준으로는 2.12 예정

 dev 뒷 숫자를 보면 알 수 있듯이, 2.6보다 더욱 최신 버전이지만 앞 2.12 부분만 다를뿐**


당연한말이겠지만 reload window 해야함!

In [38]:
'''1 INDEXING'''
# 3. SPLIT Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 한글 추천 모델
# 1. jhgan/ko-sroberta-multitask
# 2. BAAI/bge-m3
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs = {'device': 'cuda'}
)

# 4. VectorDB Store
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 20} #검색 결과 20개로 설정
)

In [39]:
'''2. Query'''
# LLM 모델 설정: huggingface free 한국어 model 활용
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# HuggingFace Model implementation
# repo_id is a registered model inside HuggingFace
repo_id = "google/gemma-2-9b-it"

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=512,
    temperature=0.1,
    huggingfacehub_api_token=hf_token
)

chat_llm = ChatHuggingFace(llm=llm_endpoint)

In [40]:
'''2. Query'''
# 쿼리단계 프롬프트 템플릿 설정
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt implementation
template = """당신은 질문-답변을 도와주는 AI 어시스턴트입니다. 
아래의 제공된 문맥(Context)을 활용해서만 질문에 답하세요. 
답을 모른다면 모른다고 말하고, 직접적인 답이 문맥에 없다면 문맥을 바탕으로 추론하지 마세요.
답변은 반드시 한국어로 작성하세요.

#Context:
{context}

#Question:
{question}

#Answer:"""

prompt = ChatPromptTemplate.from_template(template)

In [41]:
'''3. RAG 체인'''
# 3. RAG 체인 생성
# 검색기(retriever) -> 컨텍스트 전달 -> 프롬프트 적용 -> 모델 실행 -> 결과 파싱
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_llm
    | StrOutputParser()
)

In [ ]:
# 4.1 실행 테스트
question = "광주 1월 1일의 일사량을 모두 구한 이후에 평균을 구해줘"
response = rag_chain.invoke(question)

print(f"질문: {question}")
print("-" * 30)
print(f"답변: {response}")
# 틀린 답변이 나옴: 평균 일사량은 1674376.8정도의 값이 나와야 함

질문: 광주 1월 1일의 일사량을 모두 구한 이후에 평균을 구해줘
------------------------------
답변: 문맥에서 광주 1월 1일의 대기권 밖 일사량은 2375323 입니다. 





In [ ]:
# 4.1 실행 테스트
question = "광주 1월 1일의 일사량을 모두 찾아줘"
response = rag_chain.invoke(question)

print(f"질문: {question}")
print("-" * 30)
print(f"답변: {response}")
# 모두 찾아달라한거와 다른 답변이 나옴: 두개의 시간만 알아냄

질문: 광주 1월 1일의 일사량을 모두 찾아줘
------------------------------
답변: * 일자: 01-01 01:00\n지역: 광주\n위도: 35.1729\n경도: 126.8916\n대기권 밖 일사량: 0
* 일자: 01-01 21:00\n지역: 광주\n위도: 35.1729\n경도: 126.8916\n대기권 밖 일사량: 0 



